In [2]:
import pandas as pd
import psycopg2

C:\Users\marty\AppData\Local\Temp\ipykernel_4760\451024427.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
conn = psycopg2.connect(database="mydatabase",
                        host="db",
                        user="myuser",
                        password="mypassword",
                        port="5432")

OperationalError: could not translate host name "db" to address: Nieznany host. 


In [ ]:
def create_tables():
    cursor = conn.cursor()
    try:
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS links (
            movieId INT PRIMARY KEY,
            imdbId VARCHAR(10),
            tmdbId INT
        );
        """)
        
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS movies(
            movieId INT PRIMARY KEY, 
            title VARCHAR(255),
            genres TEXT
        );
        """)
        
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS ratings(
            userId INT,
            movieId INT,
            rating FLOAT,
            timestamp INT,
            PRIMARY KEY(userId, movieId)
        );
        """)
        
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS tags(
            userId INT,
            movieId INT,
            tag TEXT,
            timestamp INT,
            PRIMARY KEY (userId, movieId, tag)
        );
        """)
        conn.commit()


    except Exception as e:
        conn.rollback()
        print(f"Error occurred: {e}")
    
    finally:
        cursor.close()


In [ ]:

create_tables()

In [ ]:
def load_data(path, data):
    cursor = conn.cursor()
    df = pd.read_csv(path)

    try:
        query = f"SELECT COUNT(*) FROM {data}"
        cursor.execute(query)
        row_count = cursor.fetchone()[0]
        
        if row_count > 0:
            print(f"The {data} table already contains data.")
            return

        for i, row in df.iterrows():
            if data == 'links':
                query = "INSERT INTO links (movieId, imdbId, tmdbId) VALUES (%s, %s, %s)"
                values = (int(row['movieId']), str(row['imdbId']), int(row['tmdbId']) if not pd.isnull(row['tmdbId']) else None)
                cursor.execute(query, values)

            elif data == 'movies':
                query = "INSERT INTO movies (movieId, title, genres) VALUES (%s, %s, %s)"
                values = (int(row['movieId']), str(row['title']), str(row['genres']))
                cursor.execute(query, values)

            elif data == 'ratings':
                query = "INSERT INTO ratings (userId, movieId, rating, timestamp) VALUES (%s, %s, %s, %s)"
                values = (int(row['userId']), int(row['movieId']), float(row['rating']), int(row['timestamp']))
                cursor.execute(query, values)

            elif data == 'tags':
                query = "INSERT INTO tags (userId, movieId, tag, timestamp) VALUES (%s, %s, %s, %s)"
                values = (int(row['userId']), int(row['movieId']), str(row['tag']), int(row['timestamp']))
                cursor.execute(query, values)

        conn.commit()

    except Exception as e:
        conn.rollback()
        print(f"Error loading data into {data} table: {e}")

    finally:
        cursor.close()



In [ ]:
load_data('ml-latest-small/movies.csv', 'movies')
load_data('ml-latest-small/ratings.csv', 'ratings')
load_data('ml-latest-small/tags.csv', 'tags')
load_data('ml-latest-small/links.csv', 'links')

In [ ]:
def count_movies():
    cursor = conn.cursor()
    cursor.execute("SELECT COUNT (*) FROM MOVIES")
    movies_count = cursor.fetchone()[0]
    cursor.close()
    return movies_count


In [ ]:
count_movies()

#the number of movies is 9742

In [ ]:
def most_common_genre():
    cursor = conn.cursor()
    cursor.execute("SELECT genres FROM movies")
    rows = cursor.fetchall()

    genre_counts = {}
    for (genre,) in rows:
        genres = genre.split('|')
        for g in genres:
            g = g.strip()
            if g:
                genre_counts[g] = genre_counts.get(g, 0) + 1
    most_common = max(genre_counts.items(), key=lambda item: item[1], default=None)
    cursor.close()
    return most_common
    

In [ ]:
most_common_genre()

#the most common genre is drama with 4361 occurences

In [ ]:
def top_10_movies():
    cursor = conn.cursor()
    cursor.execute("""
    SELECT m.title, AVG(r.rating) AS average_rating
    FROM movies m
    JOIN ratings r ON m.movieId = r.movieId
    GROUP BY m.title
    ORDER BY average_rating DESC
    LIMIT 10;""")
    top_10 = cursor.fetchall()
    if top_10:
        for movie in top_10:
            title, rating = movie
            print(f"Title: {title}, Rating: {rating:.2f}")
    cursor.close()
    return top_10
    

In [ ]:
top_10_movies()

The top 10 movies are:

- Title: Brother (Brat) (1997), Rating: 5.00
- Title: Eva (2011), Rating: 5.00
- Title: Go for Zucker! (Alles auf Zucker!) (2004), Rating: 5.00
- Title: Holy Motors (2012), Rating: 5.00
- Title: Connections (1978), Rating: 5.00
- Title: Watching the Detectives (2007), Rating: 5.00
- Title: Girls About Town (1931), Rating: 5.00
- Title: Story of Women (Affaire de femmes, Une) (1988), Rating: 5.00
- Title: Nasu: Summer in Andalusia (2003), Rating: 5.00
- Title: Into the Woods (1991), Rating: 5.00



In [ ]:
def most_often_rating_users():
    cursor = conn.cursor()
    cursor.execute("""
    SELECT userId, COUNT(*) AS rating_count
    FROM ratings
    GROUP BY userId
    ORDER BY rating_count DESC
    LIMIT 5;
    """)
    top_users = cursor.fetchall()
    if top_users:
        for user in top_users:
            userid, rating_count = user
            print(f"UserId: {userid}, Rating count: {rating_count}")
    cursor.close()
    return top_users

In [ ]:
most_often_rating_users()

The most often rating users are:
- UserId: 414, Rating count: 2698
- UserId: 599, Rating count: 2478
- UserId: 474, Rating count: 2108
- UserId: 448, Rating count: 1864
- UserId: 274, Rating count: 1346

In [ ]:
def first_and_last_ratings():
    cursor = conn.cursor()
    cursor.execute("""
        SELECT m.title, TO_TIMESTAMP(r.timestamp) :: DATE AS review_date
        FROM ratings r
        JOIN movies m ON r.movieId = m.movieId;
        """)
    ratings = cursor.fetchall()
    first_rating = min(ratings, key=lambda x: x[1])
    last_rating = max(ratings, key=lambda x: x[1])
    print(f"First Rating: Title: {first_rating[0]}, Date: {first_rating[1]}")
    print(f"Last Rating: Title: {last_rating[0]}, Date: {last_rating[1]}")
    cursor.close()
    return first_rating, last_rating

In [ ]:
first_and_last_ratings()

The first and last ratings are:
- **First Rating:** 
  - Title: Copycat (1995)
  - Date: 1996-03-29

- **Last Rating:** 
  - Title: Crumb (1994)
  - Date: 2018-09-24

In [ ]:
def movies_released_in_1990():
    cursor = conn.cursor()
    try:
        cursor.execute("""
            WITH movie_years AS (
                SELECT title, 
                       REGEXP_MATCHES(title, '\\((\\d{4})\\)') AS year_matches
                FROM movies
                WHERE title LIKE '%(%'
            )
            SELECT title, CAST(year_matches[1] AS INTEGER) AS year
            FROM movie_years
            WHERE year_matches[1] IS NOT NULL AND CAST(year_matches[1] AS INTEGER) = 1990;
        """)
        
        movies_1990 = cursor.fetchall()
        
        if movies_1990:
            for movie in movies_1990:
                title, year = movie
                print(f"Title: {title}")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        cursor.close()
        
    return movies_1990



In [ ]:
movies_released_in_1990()

First 10 Movies Released in 1990

- Title: Home Alone (1990), Year: 1990
- Title: Ghost (1990), Year: 1990
- Title: Dances with Wolves (1990), Year: 1990
- Title: Pretty Woman (1990), Year: 1990
- Title: Days of Thunder (1990), Year: 1990
- Title: Grifters, The (1990), Year: 1990
- Title: Tie Me Up! Tie Me Down! (¡Átame!) (1990), Year: 1990
- Title: Paris Is Burning (1990), Year: 1990
- Title: Goodfellas (1990), Year: 1990
- Title: Trust (1990), Year: 1990